In [119]:
import re
import string
import nltk.corpus
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import warnings
from datasets import Dataset

In [120]:
warnings.filterwarnings("ignore")

In [121]:
train_df=pd.read_csv("/data/linqinhong/kaggle/tweet_disaster/train.csv",dtype={'id': np.int16, 'target': np.int8})
test_df=pd.read_csv("/data/linqinhong/kaggle/tweet_disaster/test.csv",dtype={'id': np.int16})

In [122]:
print(train_df.shape)
print(test_df.shape)

(7613, 5)
(3263, 4)


In [123]:
train_df.drop_duplicates("text",inplace=True)

In [124]:
print(train_df.shape)
print(test_df.shape)

(7503, 5)
(3263, 4)


In [125]:
train_df["text"].fillna("",inplace=True)
train_df["location"].fillna("",inplace=True)
train_df["keyword"].fillna("",inplace=True)
test_df["text"].fillna("",inplace=True)
test_df["location"].fillna("",inplace=True)
test_df["keyword"].fillna("",inplace=True)

In [126]:
train_df["text"]=train_df["keyword"]+train_df["text"]
test_df["text"]=test_df["keyword"]+test_df["text"]

In [127]:
from emot import UNICODE_EMOJI
def remove_emoji(text):
    for emot in UNICODE_EMOJI:
        text.replace(emot," ".join(UNICODE_EMOJI[emot].replace(":","").split()))
    return text


In [128]:
train_df["text"]=train_df["text"].apply(remove_emoji)
test_df["text"]=test_df["text"].apply(remove_emoji)

In [129]:
from bs4 import BeautifulSoup
def remove_html(text):
    soup=BeautifulSoup(text)
    return soup.get_text()

In [130]:
train_df["text"]=train_df["text"].apply(remove_html)
test_df["text"]=test_df["text"].apply(remove_html)

In [131]:
def remove_mention(text):
    pattern=re.compile(r"@\w+")
    return re.sub(pattern,"",text)

In [132]:
train_df["text"]=train_df["text"].apply(remove_mention)
test_df["text"]=test_df["text"].apply(remove_mention)

In [133]:
import contractions

In [134]:
train_df["text"]=train_df["text"].apply(contractions.fix)
test_df["text"]=test_df["text"].apply(contractions.fix)

In [135]:
from nltk.corpus import stopwords
def remove_stopwords(text):
    stop=set(stopwords.words("english"))
    return " ".join([word for word in str(text).split() if word not in stop])

In [136]:
train_df["text"]=train_df["text"].apply(remove_stopwords)
test_df["text"]=test_df["text"].apply(remove_stopwords)

In [137]:
def remove_url(text):
    pattern=re.compile(r"https?://(www\.)?(\w+\.)(\w+)(/\w*)?")
    return re.sub(pattern,"",text)

In [138]:
train_df["text"]=train_df["text"].apply(remove_url)
test_df["text"]=test_df["text"].apply(remove_url)

In [139]:
def remove_space(text):
    pattern=re.compile(r" +")
    return re.sub(pattern," ",text)

In [140]:
train_df["text"]=train_df["text"].apply(remove_space)
test_df["text"]=test_df["text"].apply(remove_space)

In [141]:
def remove_uni(text):
    return text.encode("ascii","ignore").decode()

In [142]:
train_df["text"]=train_df["text"].apply(remove_uni)
test_df["text"]=test_df["text"].apply(remove_uni)

In [143]:
from nltk import WordNetLemmatizer
from nltk import pos_tag
lemma=WordNetLemmatizer()
def lemmatization(text):
    return " ".join([lemma.lemmatize(word) for word in text.split()])

In [144]:
train_df["text"]=train_df["text"].apply(lemmatization)
test_df["text"]=test_df["text"].apply(lemmatization)

In [145]:
train_df["text"]=train_df["text"].str.lower()
test_df["text"]=test_df["text"].str.lower()

In [146]:
import string
def remove_punctuation(text):
    return re.sub('[%s]'%re.escape(string.punctuation),"",text)

In [147]:
train_df["text"]=train_df["text"].apply(remove_punctuation)
test_df["text"]=test_df["text"].apply(remove_punctuation)

In [148]:
from datasets import Dataset
train_dataset=Dataset.from_pandas(train_df[:7000])
val_dataset=Dataset.from_pandas(train_df[7000:])
test_dataset=Dataset.from_pandas(test_df)

In [149]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
checkpoint="bert-base-uncased"
tokenizer=AutoTokenizer.from_pretrained(checkpoint)
model=AutoModelForSequenceClassification.from_pretrained(checkpoint)
def tokenize_function(example):
  return tokenizer(example['text'], truncation=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [150]:
train_dataset=train_dataset.map(tokenize_function,batched=True)
test_dataset=test_dataset.map(tokenize_function,batched=True)
val_dataset=val_dataset.map(tokenize_function,batched=True)

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [151]:
train_dataset.column_names

['id',
 'keyword',
 'location',
 'text',
 'target',
 '__index_level_0__',
 'input_ids',
 'token_type_ids',
 'attention_mask']

In [152]:
train_dataset=train_dataset.remove_columns(['id','keyword','location','text','__index_level_0__'])
val_dataset=val_dataset.remove_columns(['id','keyword','location','text','__index_level_0__'])
test_dataset=test_dataset.remove_columns(['id','location','keyword','text'])

In [153]:
test_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3263
})

In [154]:
train_dataset=train_dataset.rename_column("target","labels")
val_dataset=val_dataset.rename_column("target","labels")

In [155]:
train_dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 7000
})

In [156]:
from transformers import DataCollatorWithPadding
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)

In [157]:
train_dataset.set_format('torch')
val_dataset.set_format('torch')
test_dataset.set_format('torch')

In [158]:
from torch.utils.data import DataLoader
train_data=DataLoader(train_dataset,shuffle=True,batch_size=64,collate_fn=data_collator)
val_data=DataLoader(val_dataset,shuffle=True,batch_size=64,collate_fn=data_collator)
test_data=DataLoader(test_dataset,shuffle=True,batch_size=8,collate_fn=data_collator)

In [159]:
for batch in train_data:
  print({k:v.shape for k, v in batch.items()})
  break

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([64]), 'input_ids': torch.Size([64, 37]), 'token_type_ids': torch.Size([64, 37]), 'attention_mask': torch.Size([64, 37])}


In [160]:
import torch

In [161]:
model=AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2,)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [162]:
from transformers import AdamW
optimizer=AdamW(model.parameters(),lr=5e-5)


In [163]:
epchoes=10
num_step=epchoes*len(train_data)
from transformers import get_scheduler
lr_scheduler=get_scheduler('linear',num_warmup_steps=0,optimizer=optimizer,num_training_steps=num_step)

In [164]:
import torch
device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [165]:
from tqdm.auto import  tqdm
import evaluate
metric=evaluate.load("accuracy")

In [166]:
device

device(type='cuda')

In [167]:
model.to(device)
progress_bar=tqdm(range(num_step))
for epoch in range(epchoes):
    model.train()
    loss=0
    for batch in train_data:
        batch={k:v.to(device) for k,v in batch.items()}
        output=model(**batch)
        loss=output.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    model.eval()
    for batch in val_data:
        batch={k:v.to(device) for k,v in batch.items()}
        output=model(**batch)
        logits=output.logits
        preds=torch.argmax(logits,dim=-1)
        metric.add_batch(predictions=preds,references=batch['labels'])
    print(metric.compute())


  0%|          | 0/1100 [00:00<?, ?it/s]

{'accuracy': 0.8190854870775348}
{'accuracy': 0.8170974155069582}
{'accuracy': 0.8151093439363817}
{'accuracy': 0.8071570576540755}
{'accuracy': 0.805168986083499}
{'accuracy': 0.8250497017892644}
{'accuracy': 0.7952286282306164}
{'accuracy': 0.8151093439363817}
{'accuracy': 0.8170974155069582}
{'accuracy': 0.8210735586481114}


In [168]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [169]:
preds=[]

In [170]:
for batch in test_data:
    batch={k:v.to(device) for k,v in batch.items()}
    output=model(**batch)
    logits=output.logits
    pred=torch.argmax(logits,dim=-1)
    preds.append(pred)

In [171]:
preds

[tensor([0, 0, 0, 1, 1, 0, 0, 1], device='cuda:0'),
 tensor([0, 0, 1, 0, 1, 1, 1, 0], device='cuda:0'),
 tensor([0, 0, 1, 1, 1, 0, 0, 1], device='cuda:0'),
 tensor([1, 0, 0, 0, 0, 0, 0, 1], device='cuda:0'),
 tensor([1, 0, 0, 0, 0, 1, 1, 0], device='cuda:0'),
 tensor([1, 1, 0, 0, 0, 1, 0, 0], device='cuda:0'),
 tensor([1, 1, 1, 0, 1, 0, 1, 0], device='cuda:0'),
 tensor([1, 0, 0, 0, 0, 1, 0, 0], device='cuda:0'),
 tensor([0, 1, 0, 0, 0, 0, 0, 0], device='cuda:0'),
 tensor([1, 0, 0, 0, 0, 0, 0, 1], device='cuda:0'),
 tensor([0, 0, 0, 1, 1, 0, 0, 0], device='cuda:0'),
 tensor([1, 0, 1, 1, 0, 1, 1, 1], device='cuda:0'),
 tensor([0, 0, 0, 0, 0, 1, 0, 0], device='cuda:0'),
 tensor([1, 0, 1, 1, 1, 0, 0, 1], device='cuda:0'),
 tensor([0, 0, 0, 1, 1, 0, 0, 0], device='cuda:0'),
 tensor([0, 0, 0, 1, 1, 1, 0, 0], device='cuda:0'),
 tensor([0, 1, 0, 0, 0, 1, 0, 0], device='cuda:0'),
 tensor([1, 0, 0, 1, 0, 1, 0, 0], device='cuda:0'),
 tensor([0, 0, 1, 1, 1, 0, 0, 0], device='cuda:0'),
 tensor([1, 

In [172]:
original_test_df=pd.read_csv('/data/linqinhong/kaggle/tweet_disaster/test.csv')

In [173]:
predictions=torch.cat([e for e in preds])

In [174]:
predictions

tensor([0, 0, 0,  ..., 0, 1, 0], device='cuda:0')

In [175]:
original_test_df['target']=predictions.to('cpu').numpy()

In [176]:
original_test_df

,id,keyword,location,text,target
0,0,NaN,NaN,Just happened a terrible car crash,0
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",0
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",0
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,1
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,1
...,...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,0
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...,0
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...,0
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...,1


In [178]:
original_test_df.drop(['text','keyword','location'],axis=1,inplace=True)
original_test_df.to_csv('/data/linqinhong/kaggle/tweet_disaster/submission.csv',index=False)

KeyError: "['text' 'keyword' 'location'] not found in axis"

In [181]:
train_df

,id,keyword,location,text,target
0,1,,,our deeds reason earthquake may allah forgive u,1
1,4,,,forest fire near la ronge sask canada,1
2,5,,,all resident asked shelter place notified offi...,1
3,6,,,13000 people receive wildfires evacuation orde...,1
4,7,,,just got sent photo ruby alaska smoke wildfire...,1
...,...,...,...,...,...
7604,10863,,,worldnews fallen powerlines glink tram update ...,1
7605,10864,,,flip side i walmart bomb everyone evacuate sta...,1
7606,10866,,,suicide bomber kill 15 saudi security site mos...,1
7608,10869,,,two giant crane holding bridge collapse nearby...,1
